In [1]:
from torch import  nn
import torch as t
from torch.nn import  functional as F

In [2]:
class ResidualBlock(nn.Module):
    '''
    实现子module: Residual Block
    '''
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
                nn.Conv2d(inchannel,outchannel,3,stride, 1,bias=False),
                nn.BatchNorm2d(outchannel),
                nn.ReLU(inplace=True),
                nn.Conv2d(outchannel,outchannel,3,1,1,bias=False),
                nn.BatchNorm2d(outchannel) )
        self.right = shortcut

    def forward(self, x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)

class ResNet(nn.Module):
    '''
    实现主module：ResNet34
    ResNet34 包含多个layer，每个layer又包含多个residual block
    用子module来实现residual block，用_make_layer函数来实现layer
    '''
    def __init__(self, num_classes=1000):
        super(ResNet, self).__init__()
        # 前几层图像转换
        self.pre = nn.Sequential(
                nn.Conv2d(3, 64, 7, 2, 3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(3, 2, 1))

        # 重复的layer，分别有3，4，6，3个residual block
        self.layer1 = self._make_layer( 64, 64, 3)
        self.layer2 = self._make_layer( 64, 128, 4, stride=2)
        self.layer3 = self._make_layer( 128, 256, 6, stride=2)
        self.layer4 = self._make_layer( 256, 512, 3, stride=2)

        #分类用的全连接
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self,  inchannel, outchannel, block_num, stride=1):
        '''
        构建layer,包含多个residual block
        '''
        shortcut = nn.Sequential(
                nn.Conv2d(inchannel,outchannel,1,stride, bias=False),
                nn.BatchNorm2d(outchannel))

        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))

        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.pre(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.avg_pool2d(x, 7)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [4]:
model = ResNet()
input  = t.randn(1, 3, 224, 224)
o = model(input)
o

tensor([[-6.5957e-02, -7.6046e-03,  4.3140e-01,  1.6750e-01, -6.2970e-01,
         -4.4403e-01,  6.0751e-01, -1.3388e-01,  3.5898e-01, -1.5403e-01,
          9.9124e-01,  8.1412e-02,  6.4325e-01,  3.5690e-01, -7.5000e-01,
         -3.1653e-01, -5.9686e-03, -1.4082e-02,  4.3914e-01,  4.1574e-01,
          1.6618e-01, -8.6653e-01, -2.9423e-01,  1.4937e-01,  1.3044e+00,
          9.8193e-01,  1.0432e+00, -2.3098e-01, -1.5814e-02, -1.5925e-01,
         -5.7309e-01,  3.2119e-02,  1.3592e+00,  1.6320e-01, -8.4363e-01,
          1.4093e-01,  1.8442e-01, -3.1257e-01, -2.0019e-01,  6.0403e-02,
         -8.7549e-01, -4.3116e-01, -2.0422e-01, -2.1505e-01, -8.1098e-01,
         -2.8208e-01, -4.5108e-01, -6.1475e-01,  8.9765e-01, -2.6048e-01,
          8.6486e-01, -4.5819e-01,  6.3399e-01,  8.3844e-03, -3.3071e-02,
         -6.7297e-02, -5.5554e-01, -1.1105e+00,  8.5416e-01, -7.8463e-02,
          6.7076e-01,  5.9399e-02,  4.1886e-01,  7.6901e-01,  2.4704e-01,
         -3.4392e-01,  1.4766e+00, -3.